In [3]:
import pandas
import numpy as np
import json
def set_pandas_display_options() -> None:
    """Set pandas display options."""
    # Ref: https://stackoverflow.com/a/52432757/
    display = pandas.options.display

    display.max_columns = 1000
    display.max_rows = 100
    display.max_colwidth = 199
    display.width = None
    display.float_format = '{:.2f}'.format
    # display.precision = 2  # set as needed
set_pandas_display_options()

In [4]:
def multiplyProbability(a,b):
    return 1 - (1 - a) * (1 - b)

def genConnections(file, servers):
    data = pandas.read_csv(file,sep=";")
    a = data.add_prefix('a_').set_index("a_to").join(data.add_prefix('b_').set_index("b_to")).reset_index()
    a = a.rename(columns={"index": "router"})
    a = a[a["a_from"] != a["b_from"]]
    a#data.set_index("to").add_prefix('c_').outerjoin(a

    helper = a["a_from"].str.startswith("w")
    two_steps = a[~helper]
    two_steps = two_steps[~two_steps["b_from"].str.startswith("w")]

    b = a[helper]
    #b = a.rename(columns={"index": "router"})
    three_steps = data.add_prefix('c_').set_index("c_to").join(b.set_index("a_from"))
    three_steps = three_steps.reset_index().rename(columns={"index": "router1", "router" : "router2"})

    three_steps = three_steps[three_steps["c_from"] != three_steps["b_from"]]
    three_steps = three_steps[three_steps["a_delay"].notnull()]
    three_steps = three_steps[~three_steps["c_from"].str.startswith("w")]

    connectionHolder = {}


    two_steps['sort_val'] = two_steps["a_from"].str[6:].astype('int32')
    two_steps['sort_val2'] = two_steps["b_from"].str[6:].astype('int32')
    two_steps = two_steps.sort_values(by=['sort_val', 'sort_val2'])

    for index, row in two_steps.iterrows():
        s1 = row["a_from"]
        s2 = row["b_from"]
        key = s1 + "-" + s2 if int(s1[6:]) < int(s2[6:]) else s2 + "-" + s1
        cValue = None
        delay = row["a_delay"] + row["b_delay"]

        if key in connectionHolder:
            cValue = connectionHolder[key]

        if cValue is None or cValue["delay"] > delay:
            rate = min(row["a_rate"], row["b_rate"])
            loss = multiplyProbability(row["a_loss"] / 100, row["b_loss"] / 100)
            corrupt = multiplyProbability(row["a_corrupt"] / 100, row["b_corrupt"] / 100)
            c = {"from" : s1, "to" : s2, "delay" : delay, "rate" : str(int(rate)) + "mbps", "loss" : loss * 100, "corrupt" : corrupt * 100}
            connectionHolder[key] = c

    three_steps['sort_val'] = three_steps["c_from"].str[6:].astype('int32')
    three_steps['sort_val2'] = three_steps["b_from"].str[6:].astype('int32')
    three_steps = three_steps.sort_values(by=['sort_val', 'sort_val2'])

    for index, row in three_steps.iterrows():
        s1 = row["c_from"]
        s2 = row["b_from"]
        key = s1 + "-" + s2 if int(s1[6:]) < int(s2[6:]) else s2 + "-" + s1
        cValue = None
        delay = row["a_delay"] + row["b_delay"] + row["c_delay"]

        if key in connectionHolder:
            cValue = connectionHolder[key]

        if cValue is None or cValue["delay"] > delay:
            rate = min(row["a_rate"], row["b_rate"], row["c_rate"])
            loss = multiplyProbability(multiplyProbability(row["a_loss"] / 100, row["b_loss"] / 100), row["c_loss"] / 100)
            corrupt = multiplyProbability(multiplyProbability(row["a_corrupt"] / 100, row["b_corrupt"] / 100), row["c_corrupt"] / 100)
            c = {"from" : s1, "to" : s2, "delay" : delay, "rate" : str(int(rate)) + "mbps", "loss" : loss * 100, "corrupt" : corrupt * 100}
            connectionHolder[key] = c
            
    for s1 in servers:
        for s2 in servers:
            if(s1 != s2):
                key = s1 + "-" + s2 if int(s1[6:]) < int(s2[6:]) else s2 + "-" + s1
                if not key in connectionHolder:
                    connectionHolder[key] = {"from" : s1, "to" : s2, "delay" : 1000, "rate" : "0mbps", "loss" : 100, "corrupt" : 100}
    return list(connectionHolder.values())               
                    
def generateStateFrom(file, statename, nextStateName, duration, servers):
    
                                  
    state = {"state_name": statename, 
            "connection_manipulation_instructions": genConnections(file, servers),
            "machine_manipulation_instructions": "reset",
            "application_instructions": [],
            "state_notifications": [],
             "transition_conditions": [
                {
                    "type": "time-based",
                    "active-for": duration,
                    "next_state": nextStateName
                }
            ]
            }
    if nextStateName is None : 
        state["transition_conditions"] = []
    return state

In [5]:
def genInfrastructure(startStateFile, servers):
    machines = []
    for s in servers:
        machines.append({
            "machine_name": s,
            "type": "t3.small",
            "image": "ami-08c148bb835696b45"
        })
        
    return {
        "aws": {
            "ec2_region": "eu-central-1",
            "ssh_key_name": "MasterThesisKey",
            "agent_port": 3100
        },
        "machines" : machines,
        "connections": genConnections(startStateFile, servers)
    }


In [6]:
def genOrchestration(filesList, servers):
     
    states = []
    s = 1
    for (file, duration) in filesList:
        thisState = "s" + str(s)
        if s == 1:
            thisState = "initial"
        states.append(generateStateFrom(file, thisState, "s" + str(s + 1) if s < len(filesList) else None, 60000 * duration, servers))
        s = s + 1


    orchestration = {
        "maximum_state_change_delay": 500, 
        "states" : states,
    }
    
    return orchestration



In [7]:
servers = ["server" + str(i) for i in range(1, 11)]    
orchestration = genOrchestration([("default.csv", 10), ("wlan2down.csv",5),("default.csv",5)], servers)
writer = open("wlan2Down.json", "w+")
json.dump(orchestration, writer)
writer.close()


infra = genInfrastructure("default.csv", servers)
writer = open("infrastructure.jsonc", "w+")
json.dump(infra, writer)
writer.close()